## 감성 사전을 만들기 위한 형용사 어간 추출

In [48]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt

In [190]:
# 어간 추출
def extract_stem(words):
    okt = Okt()
    # 어간 추출
    stems = okt.morphs(words,stem=True)
    return stems[0] if stems else None

In [5]:
df_1t = pd.read_csv('./data/review_data_1t_2.csv', encoding='utf-8')
df_1t.head(2)

,label,review,review2,adj,verb,noun,exclamation
0,희양산,[2일차: 희양산 - 대야산4山_희양산 50m© NAVER Corp.더보기 /Ope...,일차 희양산 대야산희양산 더보기 지도 데이터 지도 컨트롤러 범례부동산거리읍면동시군구...,간단한 시원한 괜찮다 어렵고 길도 험하고 불친절한 그렇지 나쁘지 많은 미끄럽 없는 ...,둘러 삼아 바라본 하면 나오는 찾아 찾아보니 앞서던 넘치고 않았던 않다 거칠어진 기...,일차 희양산 대야산 희양산 더 보기 지도 데이터 지도 컨트롤러 범례 부동산 거리 읍...,휴
1,희양산,[몇년전 부봉인가 도명산을 올랐을때 유난히 대머리가 심한 희양산이 눈에 띄어 언젠가...,몇년전 부봉인가 도명산을 올랐을때 유난히 대머리가 심한 희양산이 눈에 띄어 언젠가 ...,심한 뿌옇다 맑아진다니 간략히 희 많지 짧게 덥고 모호한데 예쁘다고 그래 있는 있 ...,올랐을 띄어 가보고 싶었는데 나면 트여 볼거란 들었다 헌데 가르는 기대는 버리지 세...,년전 부 봉인 도명 산 때 대머리 희양산 눈 한번 오늘이 그날 토요일 새벽 비 그 ...,끄응 아 엉


In [3]:
df_2t = pd.read_csv('./data/review_data_2t_2.csv', encoding='utf-8')
df_2t.head(2)

,review,label,review2,adj,verb,noun,exclamation
0,?,문화,NaN,NaN,NaN,NaN,NaN
1,친절해요,문화,친절해요,친절해요,NaN,NaN,NaN


In [6]:
df_3t = pd.read_csv('./data/review_data_3t_2.csv', encoding='utf-8')
df_3t.head(2)

,리뷰,음식유형,review2,adj,verb,noun,exclamation
0,👍,디저트,NaN,NaN,NaN,NaN,NaN
1,굿,디저트,굿,NaN,NaN,굿,NaN


In [8]:
df_1t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123983 entries, 0 to 123982
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   label        123983 non-null  object
 1   review       123983 non-null  object
 2   review2      123983 non-null  object
 3   adj          121758 non-null  object
 4   verb         122764 non-null  object
 5   noun         123833 non-null  object
 6   exclamation  30626 non-null   object
dtypes: object(7)
memory usage: 6.6+ MB


In [7]:
df_2t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141254 entries, 0 to 141253
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   review       141254 non-null  object
 1   label        141254 non-null  object
 2   review2      136766 non-null  object
 3   adj          101343 non-null  object
 4   verb         59979 non-null   object
 5   noun         93857 non-null   object
 6   exclamation  1212 non-null    object
dtypes: object(7)
memory usage: 7.5+ MB


In [9]:
df_3t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123749 entries, 0 to 123748
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   리뷰           117078 non-null  object
 1   음식유형         123749 non-null  object
 2   review2      115595 non-null  object
 3   adj          95611 non-null   object
 4   verb         64770 non-null   object
 5   noun         97115 non-null   object
 6   exclamation  1270 non-null    object
dtypes: object(7)
memory usage: 6.6+ MB


### 형용사

#### 형용사 중복 제거

In [15]:
# 중복제거전
len(' '.join(df_2t['adj'].dropna().sort_values()).split())

235178

In [29]:
# 중복제거
# 'adj'열 결측치 제거 후 오름차순 정렬후 중복제거 후 공백을 구분자로 한개의 문자열로 만든뒤 공백을 구분자로 리스트 변환
adj_list_1t = ' '.join(df_1t['adj'].dropna().sort_values().drop_duplicates()).split()
# 리스트 내의 중복 제거
adj_list_1t = list(set(adj_list_1t))
print(len(adj_list_1t))

114700


In [28]:
# 중복제거후
adj_list_2t = ' '.join(df_2t['adj'].dropna().sort_values().drop_duplicates()).split()
adj_list_2t = list(set(adj_list_2t))
print(len(adj_list_2t))

12912


In [30]:
# 중복제거후
adj_list_3t = ' '.join(df_3t['adj'].dropna().sort_values().drop_duplicates()).split()
adj_list_3t = list(set(adj_list_3t))
print(len(adj_list_3t))

11990


In [35]:
# 2,3팀 병합 후 중복제거 후 형용사 카운트
tmp=pd.concat([pd.Series(adj_list_2t),pd.Series(adj_list_3t)]).drop_duplicates()
len(tmp)

19408

In [37]:
# 1,2,3팀 병합 후 중복제거 후 형용사 카운트
tmp=pd.concat([pd.Series(adj_list_1t),pd.Series(adj_list_2t),pd.Series(adj_list_3t)]).drop_duplicates()
len(tmp)

120042

#### 형용사 어간 추출

In [168]:
from tqdm import tqdm
tqdm.pandas()
adj_stem_list_1t = pd.Series(adj_list_1t).progress_map(extract_stem)

100%|█████████████████████████████████████████████████████████████████████████| 114700/114700 [02:06<00:00, 908.95it/s]


In [177]:
# 중복제거
adj_stem_list_1t_unique = adj_stem_list_1t.drop_duplicates()
filtered_adj_stem_list_1t_unique = adj_stem_list_1t_unique[adj_stem_list_1t_unique.map(lambda x:len(x) > 1)]
filtered_adj_stem_list_1t = filtered_adj_stem_list_1t_unique.sort_values().reset_index(drop=True)
filtered_adj_stem_list_1t

0        가깝다
1       가난하다
2       가냘프다
3       가녀리다
4        가늘다
        ... 
2065     힘나다
2066     힘드다
2067     힘들다
2068     힘없다
2069     힘차다
Length: 2070, dtype: object

In [156]:
from tqdm import tqdm
tqdm.pandas()
adj_stem_list_2t = pd.Series(adj_list_2t).progress_map(extract_stem)

100%|██████████████████████████████████████████████████████████████████████████| 12912/12912 [00:11<00:00, 1137.15it/s]


In [175]:
# 중복제거
adj_stem_list_2t_unique = adj_stem_list_2t.drop_duplicates()
filtered_adj_stem_list_2t_unique = adj_stem_list_2t_unique[adj_stem_list_2t_unique.map(lambda x:len(x) > 1)]
filtered_adj_stem_list_2t = filtered_adj_stem_list_2t_unique.sort_values().reset_index(drop=True)

In [176]:
filtered_adj_stem_list_2t

0       가깝다
1      가난하다
2       가늘다
3      가능하다
4      가득하다
       ... 
988     힘나다
989     힘드다
990     힘들다
991     힘없다
992     힘차다
Length: 993, dtype: object

In [171]:
from tqdm import tqdm
tqdm.pandas()
adj_stem_list_3t = pd.Series(adj_list_3t).progress_map(extract_stem)

100%|██████████████████████████████████████████████████████████████████████████| 11990/11990 [00:11<00:00, 1046.76it/s]


In [174]:
# 중복제거
adj_stem_list_3t_unique = adj_stem_list_3t.drop_duplicates()
filtered_adj_stem_list_3t_unique = adj_stem_list_3t_unique[adj_stem_list_3t_unique.map(lambda x:len(x) > 1)]
filtered_adj_stem_list_3t = filtered_adj_stem_list_3t_unique.sort_values().reset_index(drop=True)
filtered_adj_stem_list_3t

0       가깝다
1       가늘다
2      가능하다
3      가득하다
4       가볍다
       ... 
867    희한하다
868     힘겹다
869     힘드다
870     힘들다
871     힘차다
Length: 872, dtype: object

#### 형용사 어간 파일 저장

In [235]:
tmp = pd.DataFrame(filtered_adj_stem_list_1t,columns=['단어'])
tmp['점수'] = ''
tmp.to_csv('./data/sentimental_1t_adj.csv', index=False, encoding='cp949')

In [236]:
tmp = pd.DataFrame(filtered_adj_stem_list_2t,columns=['단어'])
tmp['점수'] = ''
tmp.to_csv('./data/sentimental_2t_adj.csv', index=False, encoding='cp949')

In [237]:
tmp = pd.DataFrame(filtered_adj_stem_list_2t,columns=['단어'])
tmp['점수'] = ''
tmp.to_csv('./data/sentimental_2t_adj.csv', index=False, encoding='cp949')

In [240]:
tmp = pd.read_csv('./data/sentimental_1t_adj.csv', encoding='cp949')
tmp.head()

,단어,점수
0,가깝다,NaN
1,가난하다,NaN
2,가냘프다,NaN
3,가녀리다,NaN
4,가늘다,NaN


### 동사

#### 동사 중복 제거

In [199]:
# 중복제거후
verb_list_1t = ' '.join(df_1t['verb'].dropna().sort_values().drop_duplicates()).split()
verb_list_1t = list(set(verb_list_1t))
print(len(verb_list_1t))

297039


In [200]:
# 중복제거후
verb_list_2t = ' '.join(df_2t['verb'].dropna().sort_values().drop_duplicates()).split()
verb_list_2t = list(set(verb_list_2t))
print(len(adj_list_2t))

12912


In [201]:
# 중복제거후
verb_list_3t = ' '.join(df_3t['verb'].dropna().sort_values().drop_duplicates()).split()
verb_list_3t = list(set(verb_list_3t))
print(len(verb_list_3t))

22176


#### 동사 어간 추출

In [202]:
from tqdm import tqdm
tqdm.pandas()
verb_stem_list_1t = pd.Series(verb_list_1t).progress_map(extract_stem)

100%|█████████████████████████████████████████████████████████████████████████| 297039/297039 [05:30<00:00, 900.03it/s]


In [203]:
# 중복제거
verb_stem_list_1t_unique = verb_stem_list_1t.drop_duplicates()
filtered_verb_stem_list_1t_unique = verb_stem_list_1t_unique[verb_stem_list_1t_unique.map(lambda x:len(x) > 1)]
filtered_verb_stem_list_1t = filtered_verb_stem_list_1t_unique.sort_values().reset_index(drop=True)
filtered_verb_stem_list_1t

0          가가
1          가게
2          가계
3          가구
4          가기
        ...  
2822    힐끗거리다
2823      힘내다
2824     힘빠지다
2825      힘쓰다
2826      힘주다
Length: 2827, dtype: object

In [204]:
from tqdm import tqdm
tqdm.pandas()
verb_stem_list_2t = pd.Series(verb_list_2t).progress_map(extract_stem)

100%|██████████████████████████████████████████████████████████████████████████| 27223/27223 [00:25<00:00, 1075.10it/s]


In [205]:
# 중복제거
verb_stem_list_2t_unique = verb_stem_list_2t.drop_duplicates()
filtered_verb_stem_list_2t_unique = verb_stem_list_2t_unique[verb_stem_list_2t_unique.map(lambda x:len(x) > 1)]
filtered_verb_stem_list_2t = filtered_verb_stem_list_2t_unique.sort_values().reset_index(drop=True)
filtered_verb_stem_list_2t

0          가게
1          가기
2       가까워지다
3         가꾸다
4       가꾸어지다
        ...  
1275     흥이나다
1276     흩날리다
1277       흩다
1278      힘내다
1279      힘쓰다
Length: 1280, dtype: object

In [207]:
from tqdm import tqdm
tqdm.pandas()
verb_stem_list_3t = pd.Series(verb_list_3t).progress_map(extract_stem)

100%|██████████████████████████████████████████████████████████████████████████| 22176/22176 [00:20<00:00, 1073.96it/s]


In [213]:
# 중복제거
verb_stem_list_3t_unique = verb_stem_list_3t.drop_duplicates()
filtered_verb_stem_list_3t_unique = verb_stem_list_3t_unique[verb_stem_list_3t_unique.map(lambda x:len(x) > 1)]
filtered_verb_stem_list_3t = filtered_verb_stem_list_3t_unique.sort_values().reset_index(drop=True)
filtered_verb_stem_list_3t

0          가게
1       가까워지다
2         가꾸다
3       가꾸어지다
4         가누다
        ...  
1167       흩다
1168     흩어지다
1169      힘내다
1170      힘쓰다
1171      힘주다
Length: 1172, dtype: object

#### 동사 어간 파일 저장

In [241]:
tmp = pd.DataFrame(filtered_verb_stem_list_1t,columns=['단어'])
tmp['점수'] = ''
tmp.to_csv('./data/sentimental_1t_verb.csv', index=False, encoding='cp949')

In [242]:
tmp = pd.DataFrame(filtered_verb_stem_list_1t,columns=['단어'])
tmp['점수'] = ''
tmp.to_csv('./data/sentimental_2t_verb.csv', index=False, encoding='cp949')

In [243]:
tmp = pd.DataFrame(filtered_verb_stem_list_1t,columns=['단어'])
tmp['점수'] = ''
tmp.to_csv('./data/sentimental_3t_verb.csv', index=False, encoding='cp949')

In [244]:
tmp = pd.read_csv('./data/sentimental_1t_verb.csv', encoding='cp949')
tmp.head()

,단어,점수
0,가가,NaN
1,가게,NaN
2,가계,NaN
3,가구,NaN
4,가기,NaN


### 감탄사

#### 감탄사 중복 제거

In [221]:
# 중복제거후
exclamation_list_1t = ' '.join(df_1t['exclamation'].dropna().sort_values().drop_duplicates()).split()
exclamation_list_1t = list(set(exclamation_list_1t))
print(len(exclamation_list_1t))

162


In [222]:
# 중복제거후
exclamation_list_2t = ' '.join(df_2t['exclamation'].dropna().sort_values().drop_duplicates()).split()
exclamation_list_2t = list(set(exclamation_list_2t))
print(len(exclamation_list_2t))

63


In [223]:
# 중복제거후
exclamation_list_3t = ' '.join(df_3t['exclamation'].dropna().sort_values().drop_duplicates()).split()
exclamation_list_3t = list(set(exclamation_list_3t))
print(len(exclamation_list_3t))

61


#### 감탄사 파일저장

In [249]:
tmp = pd.DataFrame(exclamation_list_1t,columns=['단어'])
tmp['점수'] = ''
tmp.to_csv('./data/sentimental_1t_exclamation.csv', index=False, encoding='cp949')

In [250]:
tmp = pd.DataFrame(exclamation_list_2t,columns=['단어'])
tmp['점수'] = ''
tmp.to_csv('./data/sentimental_2t_exclamation.csv', index=False, encoding='cp949')

In [251]:
tmp = pd.DataFrame(exclamation_list_3t,columns=['단어'])
tmp['점수'] = ''
tmp.to_csv('./data/sentimental_3t_exclamation.csv', index=False, encoding='cp949')

In [253]:
tmp = pd.read_csv('./data/sentimental_1t_exclamation.csv', encoding='cp949')
tmp.head()

,단어,점수
0,휴,NaN
1,에헴,NaN
2,앙,NaN
3,애고,NaN
4,어머나,NaN
